In [1]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import warnings
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import gc
import os

warnings.simplefilter('ignore')
%matplotlib inline

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [2]:
seed = 2020

In [3]:
# !pip install lightgbm

In [4]:
df_feature = pd.read_pickle('data/feature.pkl')

In [5]:
df_feature.head()

,annualIncome,applicationType,delinquency_2years,dti,earliesCreditLine,employmentLength,employmentTitle,ficoRangeHigh,ficoRangeLow,grade,homeOwnership,id,initialListStatus,installment,interestRate,isDefault,issueDate,loanAmnt,n0,n1,n10,n11,n12,n13,n14,n2,n3,n4,n5,n6,n7,n8,n9,openAcc,postCode,pubRec,pubRecBankruptcies,purpose,regionCode,revolBal,revolUtil,subGrade,term,title,totalAcc,verificationStatus,issueDate_year,earliesCreditLine_year,issueDate_year_earliesCreditLine_year_minus,debt_ratio_year,applicationType_cnt,employmentLength_cnt,employmentTitle_cnt,grade_cnt,homeOwnership_cnt,initialListStatus_cnt,postCode_cnt,purpose_cnt,regionCode_cnt,subGrade_cnt,title_cnt,verificationStatus_cnt,applicationType_employmentLength_cnt,applicationType_employmentTitle_cnt,applicationType_grade_cnt,applicationType_homeOwnership_cnt,applicationType_initialListStatus_cnt,applicationType_postCode_cnt,applicationType_purpose_cnt,applicationType_regionCode_cnt,applicationType_subGrade_cnt,applicationType_title_cnt,applicationType_verificationStatus_cnt,employmentLength_applicationType_cnt,employmentLength_employmentTitle_cnt,employmentLength_grade_cnt,employmentLength_homeOwnership_cnt,employmentLength_initialListStatus_cnt,employmentLength_postCode_cnt,employmentLength_purpose_cnt,employmentLength_regionCode_cnt,employmentLength_subGrade_cnt,employmentLength_title_cnt,employmentLength_verificationStatus_cnt,employmentTitle_applicationType_cnt,employmentTitle_employmentLength_cnt,employmentTitle_grade_cnt,employmentTitle_homeOwnership_cnt,employmentTitle_initialListStatus_cnt,employmentTitle_postCode_cnt,employmentTitle_purpose_cnt,employmentTitle_regionCode_cnt,employmentTitle_subGrade_cnt,employmentTitle_title_cnt,employmentTitle_verificationStatus_cnt,grade_applicationType_cnt,grade_employmentLength_cnt,grade_employmentTitle_cnt,grade_homeOwnership_cnt,grade_initialListStatus_cnt,grade_postCode_cnt,grade_purpose_cnt,grade_regionCode_cnt,grade_subGrade_cnt,grade_title_cnt,grade_verificationStatus_cnt,homeOwnership_applicationType_cnt,homeOwnership_employmentLength_cnt,homeOwnership_employmentTitle_cnt,homeOwnership_grade_cnt,homeOwnership_initialListStatus_cnt,homeOwnership_postCode_cnt,homeOwnership_purpose_cnt,homeOwnership_regionCode_cnt,homeOwnership_subGrade_cnt,homeOwnership_title_cnt,homeOwnership_verificationStatus_cnt,initialListStatus_applicationType_cnt,initialListStatus_employmentLength_cnt,initialListStatus_employmentTitle_cnt,initialListStatus_grade_cnt,initialListStatus_homeOwnership_cnt,initialListStatus_postCode_cnt,initialListStatus_purpose_cnt,initialListStatus_regionCode_cnt,initialListStatus_subGrade_cnt,initialListStatus_title_cnt,initialListStatus_verificationStatus_cnt,postCode_applicationType_cnt,postCode_employmentLength_cnt,postCode_employmentTitle_cnt,postCode_grade_cnt,postCode_homeOwnership_cnt,postCode_initialListStatus_cnt,postCode_purpose_cnt,postCode_regionCode_cnt,postCode_subGrade_cnt,postCode_title_cnt,postCode_verificationStatus_cnt,purpose_applicationType_cnt,purpose_employmentLength_cnt,purpose_employmentTitle_cnt,purpose_grade_cnt,purpose_homeOwnership_cnt,purpose_initialListStatus_cnt,purpose_postCode_cnt,purpose_regionCode_cnt,purpose_subGrade_cnt,purpose_title_cnt,purpose_verificationStatus_cnt,regionCode_applicationType_cnt,regionCode_employmentLength_cnt,regionCode_employmentTitle_cnt,regionCode_grade_cnt,regionCode_homeOwnership_cnt,regionCode_initialListStatus_cnt,regionCode_postCode_cnt,regionCode_purpose_cnt,regionCode_subGrade_cnt,regionCode_title_cnt,regionCode_verificationStatus_cnt,subGrade_applicationType_cnt,subGrade_employmentLength_cnt,subGrade_employmentTitle_cnt,subGrade_grade_cnt,subGrade_homeOwnership_cnt,subGrade_initialListStatus_cnt,subGrade_postCode_cnt,subGrade_purpose_cnt,subGrade_regionCode_cnt,subGrade_title_cnt,subGrade_verificationStatus_cnt,title_applicationType_cnt,title_employmentLength_cnt,title_employmentTitle_cnt,title_grade_cnt,title_homeOwnership_cnt,title_initi

In [6]:
for f in df_feature.select_dtypes('object').columns:
    lbl = LabelEncoder()
    df_feature[f] = lbl.fit_transform(df_feature[f].astype(str))

In [7]:
df_train = df_feature[df_feature.isDefault.notna()].copy()
df_test = df_feature[df_feature.isDefault.isna()].copy()

df_train.shape, df_test.shape

((800000, 338), (200000, 338))

In [8]:
ycol = 'isDefault'
feature_names = list(
    filter(lambda x: x not in [ycol, 'id'], df_train.columns))

model = lgb.LGBMClassifier(objective='binary',
                           boosting_type='gbdt',
                           num_leaves=32,
                           max_depth=6,
                           learning_rate=0.05,
                           n_estimators=10000,
                           subsample=0.8,
                           feature_fraction=0.6,
                           reg_alpha=10,
                           reg_lambda=12,
                           random_state=seed,
                           is_unbalance=True,
                           metric='auc')

df_oof = df_train[['id', ycol]].copy()
df_oof['prob'] = 0
prediction = df_test[['id']]
prediction['prob'] = 0
df_importance_list = []

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
for fold_id, (trn_idx, val_idx) in enumerate(
        kfold.split(df_train[feature_names], df_train[ycol])):
    X_train = df_train.iloc[trn_idx][feature_names]
    Y_train = df_train.iloc[trn_idx][ycol]

    X_val = df_train.iloc[val_idx][feature_names]
    Y_val = df_train.iloc[val_idx][ycol]

    print('\nFold_{} Training ================================\n'.format(
        fold_id + 1))

    lgb_model = model.fit(X_train,
                          Y_train,
                          eval_names=['train', 'valid'],
                          eval_set=[(X_train, Y_train), (X_val, Y_val)],
                          verbose=100,
                          early_stopping_rounds=50)

    pred_val = lgb_model.predict_proba(
        X_val, num_iteration=lgb_model.best_iteration_)[:, 1]
    df_oof.loc[val_idx, 'prob'] = pred_val

    pred_test = lgb_model.predict_proba(
        df_test[feature_names], num_iteration=lgb_model.best_iteration_)[:, 1]
    prediction['prob'] += pred_test / kfold.n_splits

    df_importance = pd.DataFrame({
        'column': feature_names,
        'importance': lgb_model.feature_importances_,
    })
    df_importance_list.append(df_importance)

    del lgb_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
    gc.collect()


Fold_1 Training ================================

[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
Training until validation scores don't improve for 50 rounds
[100]	train's auc: 0.73966	valid's auc: 0.736327
[200]	train's auc: 0.747726	valid's auc: 0.740741
[300]	train's auc: 0.753232	valid's auc: 0.742577
[400]	train's auc: 0.757718	valid's auc: 0.743469
[500]	train's auc: 0.761737	valid's auc: 0.744012
[600]	train's auc: 0.765418	valid's auc: 0.744265
[700]	train's auc: 0.768897	valid's auc: 0.744467
[800]	train's auc: 0.772382	valid's auc: 0.744636
[900]	train's auc: 0.775656	valid's auc: 0.744668
[1000]	train's auc: 0.77886	valid's auc: 0.744679
Early stopping, best iteration is:
[1014]	train's auc: 0.779277	valid's auc: 0.74471

Fold_2 Training ================================

[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
Train

In [9]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby([
    'column'
])['importance'].agg('mean').sort_values(ascending=False).reset_index()
df_importance

,column,importance
0,issueDate,1041.8
1,revolBal,721.8
2,annualIncome,650.0
3,dti,612.0
4,debt_ratio_year,577.2
5,revolUtil,548.6
6,loanAmnt,533.6
7,installment,411.2
8,issueDate_year_earliesCreditLine_year_minus,397.8
9,totalAcc,390.2


In [10]:
auc = roc_auc_score(df_oof[ycol], df_oof['prob'])
print('auc:', auc)

auc: 0.7429533392804317


In [11]:
os.makedirs('sub', exist_ok=True)

sub = prediction[['id', 'prob']].copy()
sub.columns = ['id', 'isDefault']
sub.to_csv('sub/jiedai_{}.csv'.format(auc), index=False)

In [12]:
os.makedirs('prob', exist_ok=True)

prediction.to_csv('prob/sub_lgb.csv', index=False)
df_oof[['id', 'prob', ycol]].to_csv('prob/oof_lgb.csv', index=False)

In [13]:
df_oof.head()

,id,isDefault,prob
0,1,0.0,0.547366
1,23,1.0,0.701218
2,28,0.0,0.361765
3,32,0.0,0.644193
4,42,1.0,0.632134


In [14]:
prediction.head()

,id,prob
800000,800000,0.184450
800001,800001,0.630223
800002,800002,0.575915
800003,800003,0.616928
800004,800004,0.703135
